In [1]:
from __future__ import print_function
from __future__ import division
import os
import cPickle as pickle
import json
import subprocess
from IPython.display import display
from IPython.display import Audio

# Preprocessing CALLHOME for ZRTools


- Created: 26-Oct-2016


## Create mapping for start time for each segment

Format: Dictionary  
key: {key: value}  
*file: {file.seg.wav: start time}*  
Name: segment_start.dict, segment_start.txt  

In [2]:
def read_segments_file(seg_fname):
    segment_map = {}
    with open(seg_fname, "r") as seg_f:
        for i, line in enumerate(seg_f):
            if i == 0:
                continue
            try:
                line_items = line.strip().split()
                seg_key = line_items[0]
                file_id = line_items[1]
                if file_id not in segment_map:
                    segment_map[file_id] = {}
                seg_start = int(float(line_items[6])*100)
                segment_map[file_id][seg_key] = seg_start
            except ValueError:
                print("Incorrect line format at line: %d" % i)
    return segment_map
        

### Read segment map

In [3]:
segment_map = read_segments_file('../segments.txt')
pickle.dump(segment_map, open("../segments.dict", "wb"))

## Create VAD files for merged wavs

In [14]:
def create_merged_vad(vad_file_id, segment_map, seg_vad_path, merged_vad_path):
    total_dur_10ms = 0
    total_dur_10ms_ge500ms = 0
    with open(os.path.join(merged_vad_path, vad_file_id+".vad"), "w") as vad_f:
        print("creating vad %s ..." % vad_file_id)
        for i, (seg_id, seg_start) in enumerate(sorted(segment_map[vad_file_id].items(), key=lambda t:t[0])):
            with open(os.path.join(seg_vad_path, seg_id+".vad"), "r") as seg_vad_f:
                for line in seg_vad_f:
                    line_items = map(int, line.strip().split())
                    start = seg_start+line_items[0]
                    end = seg_start+line_items[1]
                    total_dur_10ms += (end-start)
                    total_dur_10ms_ge500ms += ((end - start) if (end-start) >= 50 else 0)
                    out_line = ("%d %d\n" %(start, end))
                    vad_f.write(out_line)
                # end for
            # end reading seg file
        # end looping over all segments
    # end writing vad file
    return total_dur_10ms, total_dur_10ms_ge500ms

### Create new directory for merged vads

In [15]:
merged_vads_path = "../mergedVads"
seg_vad_path = "../vad"
if not os.path.exists(merged_vads_path):
    os.makedirs(merged_vads_path)

### Create merged vad for each file

In [16]:
total_dur_10ms, total_dur_10ms_ge500ms = 0, 0
for vad_file_id in segment_map:
    t1, t2 = create_merged_vad(vad_file_id, segment_map, seg_vad_path, merged_vads_path)
    total_dur_10ms += t1
    total_dur_10ms_ge500ms += t2

creating vad 090 ...
creating vad 091 ...
creating vad 092 ...
creating vad 093 ...
creating vad 094 ...
creating vad 095 ...
creating vad 096 ...
creating vad 097 ...
creating vad 010 ...
creating vad 011 ...
creating vad 012 ...
creating vad 013 ...
creating vad 014 ...
creating vad 015 ...
creating vad 018 ...
creating vad 025 ...
creating vad 024 ...
creating vad 027 ...
creating vad 026 ...
creating vad 021 ...
creating vad 023 ...
creating vad 022 ...
creating vad 029 ...
creating vad 028 ...
creating vad 115 ...
creating vad 114 ...
creating vad 038 ...
creating vad 039 ...
creating vad 111 ...
creating vad 110 ...
creating vad 113 ...
creating vad 112 ...
creating vad 032 ...
creating vad 033 ...
creating vad 030 ...
creating vad 031 ...
creating vad 036 ...
creating vad 037 ...
creating vad 034 ...
creating vad 035 ...
creating vad 051 ...
creating vad 108 ...
creating vad 049 ...
creating vad 048 ...
creating vad 047 ...
creating vad 046 ...
creating vad 045 ...
creating vad 

In [19]:
print(total_dur_10ms, total_dur_10ms_ge500ms)
print(map(lambda t: "{0:.3f}".format((t / 100.0 / 3600)), [total_dur_10ms, total_dur_10ms_ge500ms]))

4724341 3091219
['13.123', '8.587']


## Create PLP features

In [35]:
merged_wavs_path = "../mergeWavs"
plp_path = "../plp"
plp_norm_path = "../std_plp"
if not os.path.exists(plp_path):
    os.makedirs(plp_path)
if not os.path.exists(plp_norm_path):
    os.makedirs(plp_norm_path)

In [8]:
with open("config.json") as json_data_file:
    config = json.load(json_data_file)

In [21]:
def create_plp(wav_fname, plp_fname):
    FEACALC = config['base']["feacalc"]
    subprocess.call([FEACALC,"-plp", \
                    "12", "-cep", "13", "-dom", "cep", "-deltaorder", \
                    "2", "-dither", "-frqaxis", "bark", "-samplerate", \
                    "8000", "-win", "25", "-step", "10", "-ip", \
                    "MSWAVE", "-rasta", "false", "-compress", \
                    "true", "-op", "swappedraw", "-o", plp_fname, wav_fname])

    
def normalize_plp(plp_fname, vad_fname, plp_norm_fname):
    STANDFEAT = config['base']["standfeat"]
    # Standardize binary file, for VAD regions only
    subprocess.call([STANDFEAT, "-D", "39", "-infile", \
                    plp_fname, "-outfile", plp_norm_fname, \
                    "-vadfile", vad_fname])

In [44]:
for i, file_id in enumerate(segment_map):
    wav_fname = os.path.join(merged_wavs_path, file_id+".wav")
    vad_fname = os.path.join(merged_vads_path, file_id+".vad")
    plp_fname = os.path.join(plp_path, file_id+".binary")
    plp_norm_fname = os.path.join(plp_norm_path, file_id+".std.binary")
    
    #print(file_id, wav_fname, vad_fname, plp_fname, plp_norm_fname)
    
    # create PLP
    if i % 20 == 0:
        print("plp for file %s " % file_id)
    
    if not os.path.exists(plp_fname):
        create_plp(wav_fname, plp_fname)
    
    if i % 20 == 0:
        print("normalizing plp %s" % file_id)
    
    if not os.path.exists(plp_norm_fname):
        normalize_plp(plp_fname, vad_fname, plp_norm_fname)
print("Completed!")

plp for file 090 
normalizing plp 090
plp for file 023 
normalizing plp 023
plp for file 051 
normalizing plp 051
plp for file 052 
normalizing plp 052
plp for file 072 
normalizing plp 072
plp for file 006 
normalizing plp 006
Completed!


## Create LSH files

In [45]:
merged_wavs_path = "../mergeWavs"
plp_path = "../plp"
plp_norm_path = "../std_plp"
lsh_path = "../lsh"
if not os.path.exists(lsh_path):
    os.makedirs(lsh_path)
lsh_proj_fname = os.path.join(lsh_path, "proj_S64xD39_seed1")

In [56]:
def create_lsh_proj_file(lsh_proj_fname):
    subprocess.call([config['base']["lsh_genproj"], \
                     "-D","39","-S","64","-seed", \
                     "1","-projfile", lsh_proj_fname])

def create_lsh_file(plp_norm_fname, vad_fname, lsh_proj_fname, lsh_fname):
    LSH = config['base']["lsh"]
    subprocess.call([LSH, "-D", "39", "-S", "64", \
                    "-projfile", lsh_proj_fname, \
                    "-featfile", plp_norm_fname, "-sigfile", \
                    lsh_fname, "-vadfile", vad_fname])

In [57]:
create_lsh_proj_file(lsh_proj_fname)
os.path.exists(lsh_proj_fname)

True

In [58]:
for i, file_id in enumerate(segment_map):
    wav_fname = os.path.join(merged_wavs_path, file_id+".wav")
    vad_fname = os.path.join(merged_vads_path, file_id+".vad")
    plp_norm_fname = os.path.join(plp_norm_path, file_id+".std.binary")
    lsh_fname = os.path.join(lsh_path, file_id+".std.lsh64")
    
    #print(file_id, wav_fname, vad_fname, plp_fname, plp_norm_fname)
    
    # create LSH
    if i % 20 == 0:
        print("lsh for file %s " % file_id)
    
    if not os.path.exists(lsh_fname):
        create_lsh_file(plp_norm_fname, vad_fname, lsh_proj_fname, lsh_fname)
        
print("Completed!")

lsh for file 090 
lsh for file 023 
lsh for file 051 
lsh for file 052 
lsh for file 072 
lsh for file 006 
Completed!


## Create ZRTools discovery command files

In [72]:
exp_path = '../exp'
if not os.path.exists(exp_path):
    os.makedirs(exp_path)

# List of wav files
wav_file_list = sorted(segment_map.keys())
exp_name = 'callhome'

In [78]:
with open(os.path.join(exp_path, 'files.base'), "w") as out_f:
    for wav_file in wav_file_list:
        out_f.write(wav_file+'\n')
print("Generated files.base")

Generated files.base


In [76]:
def create_discovery_cmd_scripts(exp_path, wav_file_list, exp_name, num_splits=1):
    disc_file_split_base = "disc_{0:d}.cmd"
    disc_file_split = os.path.join(exp_path, disc_file_split_base)
    disc_split_file = os.path.join(exp_path, "disc_split.txt")
    num_files = len(wav_file_list)
    exp_local_path = os.path.join("exp", exp_name)
    cmd_string = "scripts/plebdisc_filepair \"{0:s}\" \"{1:s}\" {2:s} 39\n"

    total_lines = num_files * num_files
    lines_per_file = total_lines // num_splits
    smallfile = None
    curr_line = 0
    curr_file_num = 0

    for i in xrange(num_files) :
        if i % 20 == 0:
            print("Progress: {0:d} out of: {1:d}".format(curr_line+1, total_lines))
        for j in xrange(num_files):
            out_line = cmd_string.format(wav_file_list[i], \
                                              wav_file_list[j], \
                                              exp_local_path)
            if curr_line % lines_per_file == 0:
                if smallfile:
                    smallfile.close()
                small_filename = disc_file_split.format(curr_file_num)
                smallfile = open(small_filename, "w")
                curr_file_num += 1
            smallfile.write(out_line)
            curr_line += 1
    if smallfile:
        smallfile.close()

    # Making a list of commands to execute the split disc list
    full_split_cmd_string = "nice sh {0:s} 1> {1:s} 2>{2:s} &\n"
    split_cmd = os.path.join(exp_local_path, "matches","{0:s}.{1:d}")
    with open(disc_split_file, "w") as out_f:
        for i in xrange(curr_file_num):
            curr_split_file = os.path.join(exp_local_path, disc_file_split_base.format(i))
            split_cmd_out = split_cmd.format("out", i)
            #split_cmd_err = split_cmd.format("err", i)
            split_cmd_err = "/dev/null"

            out_line = "nice sh "
            out_f.write(full_split_cmd_string.format(curr_split_file, \
                                                    split_cmd_out, \
                                                    split_cmd_err))

    print("Completed - disc.cmd")

In [77]:
create_discovery_cmd_scripts(exp_path=exp_path, wav_file_list=wav_file_list, exp_name=exp_name, num_splits=25)

Progress: 1 out of: 10816
Progress: 2081 out of: 10816
Progress: 4161 out of: 10816
Progress: 6241 out of: 10816
Progress: 8321 out of: 10816
Progress: 10401 out of: 10816
Completed - disc.cmd


In [63]:
map(segment_map.keys()[:5])

['090', '091', '092', '093', '094']

In [60]:
print(segment_map.keys())

['090', '091', '092', '093', '094', '095', '096', '097', '010', '011', '012', '013', '014', '015', '018', '025', '024', '027', '026', '021', '023', '022', '029', '028', '115', '114', '038', '039', '111', '110', '113', '112', '032', '033', '030', '031', '036', '037', '034', '035', '051', '108', '049', '048', '047', '046', '045', '044', '043', '042', '041', '040', '058', '059', '103', '054', '056', '057', '050', '100', '052', '053', '101', '106', '107', '104', '105', '061', '060', '063', '062', '065', '064', '067', '066', '117', '116', '076', '077', '075', '072', '073', '070', '071', '078', '079', '119', '118', '089', '088', '083', '082', '081', '087', '086', '085', '084', '002', '001', '007', '006', '005', '009', '120']
